In [1]:
from thesis.utils import *
from thesis.nets import *
from IPython.display import clear_output
import torch

In [19]:
data = view_database()
data = data[data['c_l'] == True]
data = data[data['weight_decay'] != 0]
data = data.sort_values('accuracy', ascending=False)
data_weight_decay = pd.DataFrame()

In [20]:
nets = []
for i, row in enumerate(data.iterrows()):
    if 'weight_decay' in row[1]['name']:
        data_weight_decay = data_weight_decay.append(data.iloc[i])

In [21]:
for name in data['name']:
    clear_output()
    net = load_network(name)
    w, f = net.competition_distribution()
    text = f"""tau_pos: {round(net.parameters['tau_pos'], 2)}, tau_neg: {round(net.parameters['tau_neg'], 2)},<br>
    A_pos: {round(net.parameters['A_pos'], 2)}, A_neg: {round(net.parameters['A_neg'], 2)},<br>
    weight_decay: {net.parameters['weight_decay']}"""
    f.update_layout(
        annotations=[
            dict(
                x=0,
                y=1,
                showarrow=False,
                text=text,
                xref="paper",
                yref="paper"
            )
        ])
    if not os.path.exists(f'weight_decay'):
        os.makedirs(f'weight_decay')
    f.write_image(f'weight_decay//{net.accuracy}.png')

Created LC_SNN network 9b312fe822eaa5f3abeea13f24d555964d7cdcbd349f9b84d6180b35 with parameters
{'network_type': 'LC_SNN', 'mean_weight': 0.46, 'n_iter': 4000, 'c_w': -100.0, 'c_w_min': -inf, 'time_max': 250, 'crop': 20, 'kernel_size': 12, 'kernel_prod': 144, 'stride': 4, 'n_filters': 25, 'intensity': 127.5, 'dt': 1, 'c_l': True, 'A_pos': -1.8, 'A_neg': -0.2, 'tau_pos': 13.666666666666668, 'tau_neg': 11.555555555555555, 'weight_decay': 2e-05, 'train_method': None, 'immutable_name': False}



In [2]:
n = []
a_patch = []
a_all = []
a_spikes_first = []
a_lc = []
e_patch = []
e_all = []
e_spikes_first = []
e_lc = []
f = go.FigureWidget().update_layout(
    title_text='LCSNN learning rate', height=500, width=800,
    xaxis_title_text='Number of training iterations', yaxis_title_text='Accuracy'
)
f.add_scatter(x=n, y=a_patch, error_y=dict(array=e_patch), name='patch voting', line_shape='spline')
f.add_scatter(x=n, y=a_all, error_y=dict(array=e_all), name='all voting', line_shape='spline')
f.add_scatter(x=n, y=a_spikes_first, error_y=dict(array=e_spikes_first), name='spikes first', line_shape='spline')
f.add_scatter(x=n, y=a_lc, error_y=dict(array=e_lc), name='linear classifier', line_shape='spline');

In [4]:
net = LC_SNN(n_filters=100, c_w=-120, mean_weight=0.3)

Created LC_SNN network a314ca4cd4217a855837ceb5d98c88e84dd0fd91abe284cbbf898836 with parameters
{'network_type': 'LC_SNN', 'mean_weight': 0.3, 'n_iter': 0, 'c_w': -120, 'c_w_min': -inf, 'time_max': 250, 'crop': 20, 'kernel_size': 12, 'kernel_prod': 144, 'stride': 4, 'n_filters': 100, 'intensity': 127.5, 'dt': 1, 'c_l': False, 'A_pos': None, 'A_neg': None, 'tau_pos': 20.0, 'tau_neg': 20.0, 'weight_decay': None, 'train_method': None, 'immutable_name': False}



In [5]:
net.calibrate(5000)
net.calculate_accuracy(1000, all=True, to_print=False)
n.append(net.n_iter)

a_patch.append(net.score['patch_voting']['accuracy'])
a_all.append(net.score['all_voting']['accuracy'])
a_spikes_first.append(net.score['spikes_first']['accuracy'])
a_lc.append(net.score['lc']['accuracy'])
e_patch.append(net.score['patch_voting']['error'])
e_all.append(net.score['all_voting']['error'])
e_spikes_first.append(net.score['spikes_first']['error'])
e_lc.append(net.score['lc']['error'])

f.data[0].x = n
f.data[0].y = a_patch
f.data[0].error_y = dict(array=e_patch)

f.data[1].x = n
f.data[1].y = a_all
f.data[1].error_y = dict(array=e_all)

f.data[2].x = n
f.data[2].y = a_spikes_first
f.data[2].error_y = dict(array=e_spikes_first)

f.data[3].x = n
f.data[3].y = a_lc
f.data[3].error_y = dict(array=e_lc)

torch.save(f, 'LC_SNN-bench-widget.pt')

Calibrating network...


KeyboardInterrupt: 

In [ ]:
delta_n = 500
while net.n_iter < 12000:
    clear_output()

    net.train(interval=[net.n_iter, net.n_iter + delta_n], plot=True)
    
    net.calibrate(5000)
    net.calculate_accuracy(1000, all=True, to_print=False)
    n.append(net.n_iter)

    a_patch.append(net.score['patch_voting']['accuracy'])
    a_all.append(net.score['all_voting']['accuracy'])
    a_spikes_first.append(net.score['spikes_first']['accuracy'])
    a_lc.append(net.score['lc']['accuracy'])
    e_patch.append(net.score['patch_voting']['error'])
    e_all.append(net.score['all_voting']['error'])
    e_spikes_first.append(net.score['spikes_first']['error'])
    e_lc.append(net.score['lc']['error'])

    f.data[0].x = n
    f.data[0].y = a_patch
    f.data[0].error_y = dict(array=e_patch)

    f.data[1].x = n
    f.data[1].y = a_all
    f.data[1].error_y = dict(array=e_all)

    f.data[2].x = n
    f.data[2].y = a_spikes_first
    f.data[2].error_y = dict(array=e_spikes_first)

    f.data[3].x = n
    f.data[3].y = a_lc
    f.data[3].error_y = dict(array=e_lc)
    
    torch.save(f, 'LC_SNN-bench-widget.pt')

net.save()

In [3]:
f

FigureWidget({
    'data': [{'error_y': {'array': []},
              'line': {'shape': 'spline'},
            …

In [5]:
torch.save(f, 'LC_SNN-bench-widget.pt')
# data = torch.load('LC_SNN-bench.pt')

In [4]:
to_add_n = [1500, 1750, 2000]
to_add_a_patch = [0.863, 0.868, 0.88]
to_add_a_all = [0.847, 0.85, 0.862]
to_add_a_spikes = [0.86, 0.867, 0.876]
to_add_e_patch = [0.01087341, 0.01070402, 0.01027619]
to_add_e_all = [0.0113838, 0.01129159, 0.0109067]
to_add_e_spikes = [0.01097269, 0.0107383, 0.01042228]

In [9]:
f.layout.yaxis.range = [0, 1]